# Example for PdCl2 and CuCl data

Example of routine used to find clusters inside test data and perform their identification by using classes and examples from MSARIS package

Provided example includes linear optimization search together with founding clusters and plotting plots with found spectrum

Currently for prototype runned routine is conducted mostly in user defined functions 

## Installing dependencies

Install dependencies to run package

In [ ]:
%pip install -r ../requirements.txt
%pip install -e ..

Run imports to run example scripts

In [ ]:
from collections import defaultdict
import glob
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from msaris.clusterisation.clusterisation import ClusterSearch, MaxClustering
from msaris.molecule.molecule import Molecule
from msaris.search.optimization_search import SearchClusters
from msaris.reader.reader import load_data
from msaris.reader.preprocessing import filter_intensities, reduce_data_resolution
from msaris.utils.intensities_util import norm
from msaris.utils.recognizing_utils import formal_formula

## Running data loading

Read `.mzML` file and pre-proccess data with reducing data dimensionality

In [ ]:
file_name = "PdCl2_neg_000001"
DATA_SOURCE = f"../tests/resources/{file_name}.mzML"
mz, it = load_data(
    DATA_SOURCE, range_spectrum=(100, 1500), min_intensity=None, mz_binning_width=5.0,
)
mz_processed, it_processed = reduce_data_resolution(mz, it, int(mz.shape[0]/10), int(it.shape[0]/10))

## Clustering spectrum to find existing peaks

Find MaxClustering to find clusters in data to find weighted masses with peaks

In [ ]:
clust = MaxClustering(
    window=5,
    threshold = max(it)/2000
)
masses_ = clust.find(mz_processed, it_processed)

## Running script to find brutto formula

Running parameters in order with different coefficients calculated
started from Metal to adding various ligands

In [ ]:
initial_params = dict(
    no_TBAB=True,
    no_K=True,
    no_MeOH=True,
    no_Cu=False,
    no_Pd1=False,
    no_Pd=False,
    no_NaTFA=True,
    no_OH=True,
    no_H2O=True,
    no_O2=True,
    no_O=True,
    no_N2=True,
    no_Na=True,
    no_CH3CN=True,
)
iteration_steps = dict(
    no_Cu=True,
    no_Pd=True,
    no_Na=False,
    no_CH3CN=False,
    no_NaTFA=False,
    no_H2O=False,
    no_MeOH=False,
    no_TBAB=False,
    no_O2=False,
    no_O=False,
    no_N2=False,
)

Running custom script run with found weighted masses and determing isotope pattern formulas

In [ ]:
ranked = defaultdict(list)
path = f"./data/{file_name}"

Running class with function for search
Here we used threshold for cosine value equal to 0.7

Data is saved in `ranked` dictionary with all found metrics

In [ ]:
srch = SearchClusters(
    mz=mz,
    it=it,
    charge = 1,
    threshold = 0.7,
    verbose=True
)
for target_mass in tqdm(sorted(masses_)):
    found = []
    for to_change, value in iteration_steps.items():
        params = initial_params.copy()
        params[to_change] = value
        found.extend(srch.recognise_masses(
                    target_mass,
                    params,
                    epsilon_range =(0, 5, 0.25,), 
                ))
    ranked[target_mass] = sorted(found, key=lambda x: x["delta"])  
    

## Representation of found results

Drawing general plot of the found spectra data and adding individual drawn spectrum

In [ ]:
import matplotlib as mpl

def color_fader(c1: str, c2: str, *, mix: int = 0) -> list:
    """
    Calculate fading gradient from one colour to another in RGB byte format
    
    :param c1: the first color
    :param c2: the second calor
    :param mix: int number of spectrums to get colors
    
    :return: list from colors
    """
    c1 = np.array(mpl.colors.to_rgb(c1))
    c2 = np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1 - mix) * c1 + mix * c2)

### Plotting all found peaks

Results from `ranked` are presented in total.png plot where all found peaks could be observed

In [ ]:
count_entries = len(ranked)
fig, ax = plt.subplots(1, 1, figsize=(60, 40))
max_it = max(it)
ax.plot(mz, it/ max_it, color="black")
count: int = 0
c1: str = "blue"
c2: str = "red"
for mass, data in ranked.items():
    if data:
        colour = color_fader(c1, c2, mix=count / (count_entries + 1))
        it_n = (data[0]["spectrum"][1] / max_it) * 100
        ax.plot(
            data[0]["spectrum"][0], it_n, color=colour,
        )
        max_ind = np.argmax(it_n)
        height = it_n[max_ind]
        ax.text(
            data[0]["spectrum"][0][max_ind],
            height,
            round(mass),
            color=colour,
            horizontalalignment="center",
            verticalalignment="center",
            fontsize=30,
        )
        count += 1
ax.set_title(f"Recognised masses for total spectrum", fontsize=40)
ax.set_xlabel("M/Z", fontsize=40)
ax.set_ylabel("Intensity", fontsize=40)
if not os.path.exists(path):
    os.makedirs(path)
plt.savefig(f"{path}/total.png", dpi=300)

### Individual spectrum plot

Providing indiviual spectrum found in initial spectrum

In [ ]:
for mass, data in ranked.items():
    print(f"Mass {mass} has found {len(data)}")
    for match in data:
        fig, ax = plt.subplots(1, 1, figsize=(15, 5))

        ax.plot(
            match["spectrum"][0], norm(match["spectrum"][1]) * 100, color="blue",
        )
        ax.plot(match["mz"], norm(match["it"]) * 100, color="green")
        ax.set_xlabel("M/Z", fontsize=20)
        ax.set_ylabel("Intensity", fontsize=20)

        test_label = [
            f"{formal_formula(match['composition'])}",
            f"Delta m/z: {match['delta']:.3f}",
            f"Cosine: {match['metrics']['cosine']:.3f}",
            f"Relative: {match['relative']:.3f}",
        ]

        ax.text(
            0.8,
            0.8,
            "\n".join(test_label),
            color="black",
            horizontalalignment="center",
            verticalalignment="center",
            fontsize=12,
            transform=ax.transAxes,
        )
        ax.set_title(f"Plotting relative intensity for {int(mass+0.5)}", fontsize=20)
        cluster_path = f"{path}/{int(mass+0.5)}"
        if not os.path.exists(cluster_path):
            os.makedirs(cluster_path)
        plt.savefig(f"{cluster_path}/{match['formula']}.png", dpi=300)

### Generating csv report with total found data

Performing construction of the `.csv` file with defined and found peaks

In [ ]:
report = {
    "mass": [],
    "brutto": [],
    "brutto_formal": [],
    "cosine": [],
    "relative": [],
    "delta": []
}
variables = [
    "Pd1",
    "Pd2",
    "Cu1",
    "Cu2",
    "Na",
    "K",
    "O",
    "CF3COO",
    "TBAB",
    "OH",
    "Cl",
    "Br",
    "CH3CN",
    "CH3OH",
    "H2O",
    "N2",
    "O2_1",
    "O2_2"
]
for variable in variables:
    report[variable] = []
for mass, data in ranked.items():

    for match in data:
        composition = match["composition"]
        report["mass"].append(round(mass))
        report["brutto"].append(match["formula"])
        report["brutto_formal"].append(formal_formula(composition))
        report["cosine"].append(match["metrics"]["cosine"])
        report["relative"].append(match["relative"])
        report["delta"].append(match["delta"])
        for variable in variables:
            if variable in composition:
                report[variable].append(composition[variable])
            else:
                report[variable].append(0)
    pd.DataFrame(report).to_csv(f"{path}/{file_name}.csv")